In [2]:
import pandas as pd
import numpy as np
import re
import sklearn
from sklearn import cluster
import matplotlib.pyplot as plt
import seaborn as sb
import matplotlib as g
from sklearn.preprocessing import OneHotEncoder
import random

In [10]:
# Reading the Data

In [11]:
movie2 = pd.read_csv("/Users/michael/Documents/UCSB_2017-2018/PSTAT 134/134MHCP/cleaned_movies.csv", encoding='utf-8')
movie2.head()

,Unnamed: 0,Title,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,0,#1 Cheerleader Camp (2010),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,#chicagoGirl: The Social Network Takes on a Di...,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2,#Horror (2015),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,#Pellichoopulu (2016),0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
4,4,"$1,000 on the Black (1966)",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [3]:
movie = pd.read_pickle('/Users/michael/Documents/UCSB_2017-2018/PSTAT 134/134MHCP/cleaned_movie_set.pkl')

In [5]:
movie.head()

,Title,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,#1 Cheerleader Camp (2010),0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,#chicagoGirl: The Social Network Takes on a Di...,1,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,#Horror (2015),0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,#Pellichoopulu (2016),0,0,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
4,"$1,000 on the Black (1966)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# Calculating Genre Correlations

In [12]:
df = movie
c = df.corr().abs()
s = c.unstack()
so = s.sort_values(ascending=False, kind="quicksort")
genre_corr = so[28::2]
genre_corr[0:28]

Unnamed: 0   Unnamed: 0     1.000000
Documentary  Drama          0.274506
Animation    Short          0.260361
Action       Adventure      0.246525
Drama        Comedy         0.228739
Horror       Drama          0.223028
Adventure    Animation      0.215504
Comedy       Thriller       0.214372
Animation    Family         0.209395
Film-Noir    Crime          0.186115
Horror       Thriller       0.184386
Sci-Fi       Horror         0.172706
Biography    Documentary    0.171134
             History        0.169102
Animation    Drama          0.162567
Mystery      Thriller       0.160600
Comedy       Documentary    0.160102
Crime        Action         0.157357
Sci-Fi       Drama          0.149632
Drama        Adventure      0.148040
Adventure    Family         0.147692
Mystery      Horror         0.144292
News         Documentary    0.140732
Music        Documentary    0.135527
Romance      Documentary    0.132402
             Comedy         0.129779
Drama        Romance        0.128822
A

In [13]:
counts = [] #getting the counts per genre
genres = df.columns[2::]
for i in genres:
    counts.append(df[i].value_counts().to_dict())

In [15]:
#creating counts table
counts_df = pd.DataFrame.from_dict(counts)

counts_df[2] = genres

counts_table = counts_df.drop([0], axis=1)

counts_table = counts_table.rename(columns = {1:'counts', 2:'genre'})
counts_sort = counts_table.sort_values('counts', ascending=False)

#creating column of cumulative sums
sumcol = counts_sort['counts'].sum()
cumsum = counts_sort['counts']/sumcol
counts_sort['cumsum'] = cumsum
counts_sort

,counts,genre,cumsum
8,18168,Drama,0.231063
5,11490,Comedy,0.146131
20,5694,Romance,0.072417
0,4961,Action,0.063095
6,4854,Crime,0.061734
25,4487,Thriller,0.057066
14,3770,Horror,0.047947
2,3518,Adventure,0.044742
7,3343,Documentary,0.042517
17,2223,Mystery,0.028272


# acquiring all column labels we decided we dont want

In [16]:
garb = counts_sort['genre'].where(counts_sort['cumsum']<.04).tolist()
garb = garb[9::]
garb

['Mystery',
 'Family',
 'Sci-Fi',
 'Fantasy',
 'Biography',
 'Animation',
 'History',
 'Music',
 'War',
 'Short',
 'Western',
 'Musical',
 'Sport',
 'Film-Noir',
 'News',
 'Adult',
 'Talk-Show',
 'Reality-TV',
 'Game-Show']

In [30]:
clean_df = df #clean dataset without garbage labels
clean_df = clean_df.drop(garb, axis=1) #removing the unwanted genres from our dataset
clean_df = clean_df.drop(clean_df.columns[0], axis=1)
clean_df.head()

,Title,Action,Adventure,Comedy,Crime,Documentary,Drama,Horror,Romance,Thriller
0,#1 Cheerleader Camp (2010),0,0,1,0,0,0,0,0,0
1,#chicagoGirl: The Social Network Takes on a Di...,1,0,0,0,1,1,0,0,0
2,#Horror (2015),0,0,0,0,0,1,1,0,0
3,#Pellichoopulu (2016),0,0,1,0,0,1,0,1,0
4,"$1,000 on the Black (1966)",0,0,0,0,0,0,0,0,0


In [18]:
#New Correlations after removing excess labels
c = clean_df.corr().abs()
s = c.unstack()
so = s.sort_values(ascending=False, kind="quicksort")
genre_corr = so[9::2]
genre_corr[0:12]

Documentary  Drama          0.274506
Action       Adventure      0.246525
Comedy       Drama          0.228739
Drama        Horror         0.223028
Thriller     Comedy         0.214372
             Horror         0.184386
Comedy       Documentary    0.160102
Action       Crime          0.157357
Drama        Adventure      0.148040
Romance      Documentary    0.132402
Comedy       Romance        0.129779
Drama        Romance        0.128822
dtype: float64

In [19]:
#combining adventure and action
Action_Adventure=[]
for i in range(0, len(clean_df), 1):
    if (clean_df['Action'][i]==1 or clean_df['Adventure'][i]==1):
        Action_Adventure.append(1)
    else:
        Action_Adventure.append(0)
        
#combining Drama and Documentary
Drama_Documentary=[]
for i in range(0, len(clean_df), 1):
    if (clean_df['Drama'][i]==1 or clean_df['Documentary'][i]==1):
        Drama_Documentary.append(1)
    else:
        Drama_Documentary.append(0)

In [20]:
no_titles = clean_df.drop(["Title"], axis=1) #final_df without the Title columns (final "raw" dataset before undersampling)
clean_df["Action/Adventure"] = Action_Adventure
clean_df["Drama/Documentary"] = Drama_Documentary
final_df = clean_df.drop(['Action', 'Adventure', 'Documentary', 'Drama'], axis=1)
final_df

,Title,Comedy,Crime,Horror,Romance,Thriller,Action/Adventure,Drama/Documentary
0,#1 Cheerleader Camp (2010),1,0,0,0,0,0,0
1,#chicagoGirl: The Social Network Takes on a Di...,0,0,0,0,0,1,1
2,#Horror (2015),0,0,1,0,0,0,1
3,#Pellichoopulu (2016),1,0,0,1,0,0,1
4,"$1,000 on the Black (1966)",0,0,0,0,0,0,0
5,"$100,000 for Ringo (1965)",0,0,0,0,0,0,1
6,$5 a Day (2008),1,0,0,0,0,0,1
7,$50K and a Call Girl: A Love Story (2014),1,0,0,0,0,1,0
8,$9.99 (2008),0,0,0,0,0,0,1
9,$ellebrity (2012),0,0,0,0,0,0,1


In [22]:
#this cell acquires the new cumulative sums after we merging some labels
clean_counts = []
clean_genre = no_titles.columns.tolist()
#clean_genre = final_df.columns.tolist()
for i in clean_genre:
    clean_counts.append(clean_df[i].value_counts().to_dict())
    
c_counts_df = pd.DataFrame.from_dict(clean_counts)
c_counts_df

c_counts_df[2] = clean_genre
c_counts_df
c_counts_table = c_counts_df.drop([0], axis=1)

c_counts_table = c_counts_table.rename(columns = {1:'counts', 2:'genre'})
c_counts_sort = c_counts_table.sort_values('counts', ascending=False)

c_sumcol = c_counts_sort['counts'].sum()
c_cumsum = c_counts_sort['counts']/c_sumcol
c_counts_sort['cumsum'] = c_cumsum
c_counts_sort

,counts,genre,cumsum
5,18168,Drama,0.301368
2,11490,Comedy,0.190595
7,5694,Romance,0.094451
0,4961,Action,0.082292
3,4854,Crime,0.080518
8,4487,Thriller,0.074430
6,3770,Horror,0.062536
1,3518,Adventure,0.058356
4,3343,Documentary,0.055453


# Downsampling!

In [24]:
sampled = final_df #duplicate our dataset to work with
sampled['sample']=0 #append a column of zeros

In [25]:
#randomly select 4000 observations from each genre without replacment
random.seed(1)
for i in clean_genre:
    i = np.random.choice(no_titles.where(no_titles[i]==1).index, size=4000, replace=False)
    sampled['sample'][i]=1 #changing the sample value to 1 means we will 'select' this observation
sampled

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Title,Comedy,Crime,Horror,Romance,Thriller,Action/Adventure,Drama/Documentary,sample
0,#1 Cheerleader Camp (2010),1,0,0,0,0,0,0,1
1,#chicagoGirl: The Social Network Takes on a Di...,0,0,0,0,0,1,1,1
2,#Horror (2015),0,0,1,0,0,0,1,1
3,#Pellichoopulu (2016),1,0,0,1,0,0,1,1
4,"$1,000 on the Black (1966)",0,0,0,0,0,0,0,1
5,"$100,000 for Ringo (1965)",0,0,0,0,0,0,1,1
6,$5 a Day (2008),1,0,0,0,0,0,1,1
7,$50K and a Call Girl: A Love Story (2014),1,0,0,0,0,1,0,1
8,$9.99 (2008),0,0,0,0,0,0,1,0
9,$ellebrity (2012),0,0,0,0,0,0,1,0


In [26]:
undersampled = sampled.loc[sampled['sample']==1] #only keep observations where sample=1
undersampled = undersampled.drop(['sample'], axis=1) #delete sample column, since they will all be 1
undersampled

,Title,Comedy,Crime,Horror,Romance,Thriller,Action/Adventure,Drama/Documentary
0,#1 Cheerleader Camp (2010),1,0,0,0,0,0,0
1,#chicagoGirl: The Social Network Takes on a Di...,0,0,0,0,0,1,1
2,#Horror (2015),0,0,1,0,0,0,1
3,#Pellichoopulu (2016),1,0,0,1,0,0,1
4,"$1,000 on the Black (1966)",0,0,0,0,0,0,0
5,"$100,000 for Ringo (1965)",0,0,0,0,0,0,1
6,$5 a Day (2008),1,0,0,0,0,0,1
7,$50K and a Call Girl: A Love Story (2014),1,0,0,0,0,1,0
10,'71 (2014),0,0,0,0,1,1,1
12,'Doc' (1971),0,0,0,0,0,0,0


In [27]:
undersampled.to_pickle('Undersampled Dataset')

In [42]:
####################################################################################
##  if we only want to use the top 6 genres without grouping, use this dataframe  ##
####################################################################################
sixgenres = clean_df.drop(['Adventure', 'Documentary', 'Drama'], axis=1)
sixgenres['sample']=0
#randomly select 4000 observations from each genre without replacment
# random.seed(1)
# for i in sixgenres.columns[1::]:
#     i = np.random.choice(sixgenres.where(sixgenres[i]==1).index, size=4000, replace=False)
#     sixgenres['sample'][i]=1 #changing the sample value to 1 means we will 'select' this observation
# sixgenres = sixgenres.loc[sixgenres['sample']==1] #only keep observations where sample=1
# sixgenres = sixgenres.drop(['sample'], axis=1) #delete sample column, since they will all be 1
sixgenres.head()

,Title,Action,Comedy,Crime,Horror,Romance,Thriller,sample
0,#1 Cheerleader Camp (2010),0,1,0,0,0,0,0
1,#chicagoGirl: The Social Network Takes on a Di...,1,0,0,0,0,0,0
2,#Horror (2015),0,0,0,1,0,0,0
3,#Pellichoopulu (2016),0,1,0,0,1,0,0
4,"$1,000 on the Black (1966)",0,0,0,0,0,0,0


In [45]:
col_list = ['Action','Comedy','Crime','Horror','Romance','Thriller']

In [46]:
sixgenres['Empty'] = sixgenres[col_list].sum(axis = 1)

In [48]:
df = sixgenres[sixgenres.Empty != 0]

In [52]:
df.head()

,Title,Action,Comedy,Crime,Horror,Romance,Thriller,sample,Empty
0,#1 Cheerleader Camp (2010),0,1,0,0,0,0,0,1
1,#chicagoGirl: The Social Network Takes on a Di...,1,0,0,0,0,0,0,1
2,#Horror (2015),0,0,0,1,0,0,0,1
3,#Pellichoopulu (2016),0,1,0,0,1,0,0,2
6,$5 a Day (2008),0,1,0,0,0,0,0,1


In [55]:
for i in df.columns[1:]:
    print(i)
    print(df[i].sum())
    

Action
4961
Comedy
11490
Crime
4854
Horror
3770
Romance
5694
Thriller
4487
sample
0
Empty
35256


In [56]:
comedy = df[df['Comedy'] == 1]

In [62]:
np.random.seed = 1
b = np.random.choice(comedy.Title,size = 4000, replace = False)

In [69]:
df2 = comedy[comedy['Title'].isin(b)]

In [70]:
df2.head()

,Title,Action,Comedy,Crime,Horror,Romance,Thriller,sample,Empty
31,(500) Days of Summer (2009),0,1,0,0,1,0,0,2
36,*batteries not included (1987),0,1,0,0,0,0,0,1
67,10 Rules for Sleeping Around (2013),0,1,0,0,1,0,0,2
68,10 Things I Hate About You (1999),0,1,0,0,1,0,0,2
70,10 Years (2011),0,1,0,0,1,0,0,2


In [76]:
df2.Title

31                             (500) Days of Summer (2009)
36                          *batteries not included (1987)
67                     10 Rules for Sleeping Around (2013)
68                       10 Things I Hate About You (1999)
70                                         10 Years (2011)
79                                        10.000 Km (2014)
82                                 100 Bloody Acres (2012)
88                        100 gramm dlya khrabrosti (1977)
92                                100 Years of Evil (2010)
95                                        100% Love (2011)
100                                  101 Dalmatians (1961)
102                                  102 Dalmatians (2000)
125                                     12 in a Box (2007)
131                                      12 Storeys (1997)
135                                           12:01 (1993)
158                                  14 Going on 30 (1988)
169                                         16-Love (201

In [91]:
df3 = df[(df['Title'].isin(df2.Title) & df['Comedy'] == 1) | (df['Comedy'] == 0)]

In [82]:
df3.head(50)

,Title,Action,Comedy,Crime,Horror,Romance,Thriller,sample,Empty
1,#chicagoGirl: The Social Network Takes on a Di...,1,0,0,0,0,0,0,1
2,#Horror (2015),0,0,0,1,0,0,0,1
10,'71 (2014),1,0,0,0,0,1,0,2
11,'D' (2005),0,0,1,0,0,1,0,2
14,'G' Men (1935),0,0,1,0,0,0,0,1
17,'I Know Where I'm Going!' (1945),0,0,0,0,1,0,0,1
22,'Pimpernel' Smith (1941),0,0,0,0,0,1,0,1
23,'R Xmas (2001),0,0,1,0,0,0,0,1
25,"'Sheba, Baby' (1975)",1,0,1,0,0,0,0,2
31,(500) Days of Summer (2009),0,1,0,0,1,0,0,2


In [81]:
for i in df3.columns[1:]:
    print(i)
    print(df3[i].sum())

Action
4292
Comedy
4000
Crime
4150
Horror
3414
Romance
3995
Thriller
4341
sample
0
Empty
24192


In [83]:
len(df3.Title)

17444

In [92]:
df4 = df3.drop(df3.columns[-2:],axis= 1)

In [93]:
df4.head()

,Title,Action,Comedy,Crime,Horror,Romance,Thriller
1,#chicagoGirl: The Social Network Takes on a Di...,1,0,0,0,0,0
2,#Horror (2015),0,0,0,1,0,0
10,'71 (2014),1,0,0,0,0,1
11,'D' (2005),0,0,1,0,0,1
14,'G' Men (1935),0,0,1,0,0,0


In [94]:
df4.to_pickle('Top6.pkl')